In [1]:
#basic
import numpy as np 
import pandas as pd
import time
import os

#NN
import torch
def format_pytorch_version(version):
  return version.split('+')[0]

from torch_geometric.data import Data, Dataset#,DataLoader
from torch_geometric.loader import DataLoader
from torch import Tensor
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch_geometric.transforms as Tr
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import to_networkx
from torch.nn import Sequential as Seq, Linear, ReLU, Sigmoid
from torch.optim.lr_scheduler import StepLR
from collections import namedtuple
from torch_scatter import scatter_add
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")
from utils import GraphDataset, get_pt_files, make_mlp, binary_acc

## Dataset and Dataloader

In [2]:
######################READ FILES##############
input_dir = "/global/cfs/cdirs/m3443/data/EventFilterGNN/InitialExplore"

graph_files_train = get_pt_files(input_dir, subdir="Train")
graph_files_val = get_pt_files(input_dir, subdir="Val")
graph_files_test = get_pt_files(input_dir, subdir="Test")



In [3]:
#####################GET THE DATALOADER FOR EACH SET##############
params = {'batch_size': 32, 'shuffle': True}
train_set = GraphDataset(graph_files_train)
train_loader = DataLoader(train_set, **params)

val_set = GraphDataset(graph_files_val)
val_loader = DataLoader(val_set, **params)

test_set = GraphDataset(graph_files_test)
test_loader = DataLoader(test_set, **params)


## NN Architecture

In [4]:
class RelationalModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(RelationalModel, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, m):
        return self.layers(m)

class ObjectModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(ObjectModel, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, C):
        return self.layers(C)


class InteractionNetwork(MessagePassing):
    def __init__(self, node_f_size, edge_attr_size,message_out, update_out, hidden_size):
        super(InteractionNetwork, self).__init__(aggr='add', 
                                                 flow='source_to_target')
        self.R1 = RelationalModel(2*node_f_size + edge_attr_size, message_out, hidden_size)    # 19 is the node_features * 2 + edge atributes output 4 
        self.O = ObjectModel(node_f_size + message_out, update_out, hidden_size)    # 10 is node features + output R1
        self.R2 = RelationalModel(2*update_out + message_out , 1, hidden_size)  #10 is from 2* output O + output R1(from the concat) 
        self.E: Tensor = Tensor()

    def forward(self, x: Tensor, edge_index: Tensor, edge_attr: Tensor) -> Tensor:
        x_tilde = self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)
        m2 = torch.cat([x_tilde[edge_index[1]],
                        x_tilde[edge_index[0]],
                        self.E], dim=1)
        return torch.sigmoid(self.R2(m2))
        
    def message(self, x_i, x_j, edge_attr):
        m1 = torch.cat([x_i, x_j, edge_attr], dim=1)
        self.E = self.R1(m1)
        return self.E

    def update(self, aggr_out, x):
        c = torch.cat([x, aggr_out], dim=1)
        return self.O(c) 

In [4]:
class InteractionNetwork(nn.Module):

    """
    An interaction network class
    """

    def __init__(self, hparams):
        super(InteractionNetwork, self).__init__()
        """
        Initialise the Lightning Module that can scan over different GNN training regimes
        """
        self.hparams = hparams

        # Setup input network
        self.node_encoder = make_mlp(
            hparams["num_features"],
            [hparams["hidden"]] * hparams["nb_node_layer"],
            output_activation=None,
        )

        # The edge network computes new edge features from connected nodes
        self.edge_encoder = make_mlp(
            2 * (hparams["hidden"]),
            [hparams["hidden"]] * hparams["nb_edge_layer"],
            output_activation=None
        )

        # The edge network computes new edge features from connected nodes
        self.edge_network = make_mlp(
            3 * hparams["hidden"],
            [hparams["hidden"]] * hparams["nb_edge_layer"],
            output_activation=None
        )

        # The node network computes new node features
        self.node_network = make_mlp(
            2 * hparams["hidden"],
            [hparams["hidden"]] * hparams["nb_node_layer"],
            output_activation=None
        )

        # Final edge output classification network
        self.output_edge_classifier = make_mlp(
            3 * hparams["hidden"],
            [hparams["hidden"]] * hparams["nb_edge_layer"] + [1],
            output_activation=None
        )


    def message_step(self, x, start, end, e):

        aggr_nodes = scatter_add(e, end, dim=0, dim_size=x.shape[0])
        node_inputs = torch.cat([x, aggr_nodes], dim=-1)

        x_tilde = self.node_network(node_inputs)

        x_tilde = x_tilde + x

        # Compute new edge features
        edge_inputs = torch.cat([x_tilde[start], x_tilde[end], e], dim=-1)
        e_tilde = self.edge_network(edge_inputs)

        e_tilde += e_tilde + e

        return x_tilde, e_tilde

    def output_step(self, x, start, end, e):

        classifier_inputs = torch.cat([x[start], x[end], e], dim=1)

        return self.output_edge_classifier(classifier_inputs)

    def forward(self, x, edge_index):

        start, end = edge_index

        # Encode the graph features into the hidden space
        x_enc = self.node_encoder(x)
        e_enc = self.edge_encoder(torch.cat([x_enc[start], x_enc[end]], dim=1))

        # Loop over iterations of edge and node networks
        for _ in range(self.hparams["n_graph_iters"]):

            x_enc, e_enc = self.message_step(x_enc, start, end, e_enc)

        return self.output_step(x_enc, start, end, e_enc)

## Version 2

In [5]:
preccision_acc = 0.5
def train( model, device, train_loader, optimizer, epoch, hparams):
    """train model"""
    model.train()
    times_list = []
    losses = []
    accs = []
    epoch_t0 = time()
    for batch_idx, data in tqdm(enumerate(train_loader)):
        batch = data.to(device)
        optimizer.zero_grad()
        output = model(batch.x, batch.edge_index).squeeze()
        loss = F.binary_cross_entropy_with_logits(output, batch.y, pos_weight=torch.tensor(hparams["weight"]))
        loss.backward()
        optimizer.step()
        losses.append(loss.item())   
        output = torch.sigmoid(output)
        accs.append(binary_acc(y_pred = output, y_test = batch.y, thld = preccision_acc ))
    times = time()-epoch_t0
    times_list.append(times)
    print(f"...epoch time: {times}s")
    print(f"...epoch {epoch}: mean train loss={np.mean(losses):.6f}")
    return losses, accs,times_list

def validate(model, device, val_loader, hparams):
    """validates model, returns optimal treshold"""
    model.eval()
    eff_list, pur_list, auc_list = [], [], []
    for data in val_loader:
        batch = data.to(device)
        output = model(batch.x, batch.edge_index).squeeze()
        loss = F.binary_cross_entropy_with_logits(output, batch.y, pos_weight=torch.tensor(hparams["weight"])).item()
        output = torch.sigmoid(output)

        # calculate accuracy
        eff, pur, auc = calculate_metrics(y_pred = output, y_true = batch.y)

    print(f"val eff={eff:.6f}......val pur={pur:.6f}......val auc={auc:.6f}......val loss={loss:.6f}")

def calculate_metrics(y_pred, y_true):
    """calculate metrics"""
    tp = ((y_pred > preccision_acc) & (y_true == 1)).sum().item()
    t = (y_true == 1).sum().item()
    p = (y_pred > preccision_acc).sum().item()

    eff = tp/(t+1e-6)
    pur = tp/(p+1e-6)
    auc = roc_auc_score(y_true.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
    return eff, pur, auc

In [6]:
EPOCH = 400
hparams = {
    "num_features": 6,
    "hidden": 64,
    "nb_node_layer": 3,
    "nb_edge_layer": 3,
    "n_graph_iters": 1,
    "weight": 3.,
}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = InteractionNetwork(hparams).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=30,
                       gamma=0.8) 
                         
from time import time
output = {'train_loss': [], 'test_loss': [], 'test_acc': []}
losses_l, accs_l,times_l = [],[],[]
for epoch in range(1, EPOCH + 1):
  print("---- Epoch {} ----".format(epoch))
  losses, accs,times = train( model, device, train_loader, optimizer, epoch, hparams)
  losses_l.append(losses)
  accs_l.append(accs)
  times_l.append(times)
  validate(model, device, val_loader, hparams)
  scheduler.step()

cuda:0
---- Epoch 1 ----


20it [00:05,  3.91it/s]


...epoch time: 5.112700939178467s
...epoch 1: mean train loss=0.626861
val eff=0.000000......val pur=0.000000......val auc=0.520219......val loss=0.595662
---- Epoch 2 ----


20it [00:04,  4.94it/s]


...epoch time: 4.052841424942017s
...epoch 2: mean train loss=0.609894
val eff=0.000000......val pur=0.000000......val auc=0.506625......val loss=0.602673
---- Epoch 3 ----


...

...epoch time: 4.070743083953857s
...epoch 272: mean train loss=0.118843
val eff=0.888177......val pur=0.688859......val auc=0.988684......val loss=0.125929
---- Epoch 273 ----


20it [00:03,  5.05it/s]


...epoch time: 3.963681936264038s
...epoch 273: mean train loss=0.118743
val eff=0.899767......val pur=0.673823......val auc=0.988051......val loss=0.131134
---- Epoch 274 ----


0it [00:00, ?it/s]


KeyboardInterrupt: 

## Version 1

In [18]:

###############################TUNABLE PARAMETERS###############
#define the size of layers on the nn 
hidden_l_size = 16   #tunable parameter
message_out = 4       #tunable parameter
update_out = 3        #tunable parameter
edge_attr_size = train_set.get(0).edge_attr.shape[1]
node_f_size = train_set.get(0).x.shape[1]
#initialize model 
model = InteractionNetwork(node_f_size = node_f_size, edge_attr_size = edge_attr_size,message_out = message_out, update_out= update_out,hidden_size = hidden_l_size).to(device)
#MORE TUNABLE PARAMETERS
optimizer = optim.Adam(model.parameters(), lr=0.005)
scheduler = StepLR(optimizer, step_size=10,
                       gamma=0.9)               #https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.StepLR.html

In [19]:
print(node_f_size)

6


## Training model

In [22]:
preccision_acc = 0.5
def train( model, device, train_loader, optimizer, epoch):
    """train model"""
    model.train()
    times_list = []
    losses = []
    accs = []
    epoch_t0 = time()
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.edge_attr)
        y, output = data.y, output.squeeze(1)
        loss = F.binary_cross_entropy(output, y, reduction='mean')
        loss.backward()
        optimizer.step()
      
        losses.append(loss.item())   
        accs.append(binary_acc(y_pred = output, y_test = y, thld = preccision_acc ))
    times = time()-epoch_t0
    times_list.append(times)
    print(f"...epoch time: {times}s")
    print(f"...epoch {epoch}: mean train loss={np.mean(losses):.6f}......train acc={np.mean(accs):.6f}")
    return losses, accs,times_list

def validate(model, device, val_loader):
    """validates model, returns optimal treshold"""
    model.eval()
    opt_thlds, accs = [], []
    for batch_idx, data in enumerate(val_loader):
        data = data.to(device)
        output = model(data.x, data.edge_index, data.edge_attr)
        y, output = data.y, output.squeeze()
        loss = F.binary_cross_entropy(output, y, reduction='mean').item()
        
        # define optimal threshold (thld) where TPR = TNR 
        diff, opt_thld, opt_acc = 100, 0, 0
        best_tpr, best_tnr = 0, 0
        for thld in np.arange(0.001, 0.5, 0.001):
            TP = torch.sum((y==1) & (output>thld)).item()
            TN = torch.sum((y==0) & (output<thld)).item()
            FP = torch.sum((y==0) & (output>thld)).item()
            FN = torch.sum((y==1) & (output<thld)).item()
            acc = (TP+TN)/(TP+TN+FP+FN)
            TPR, TNR = TP/(TP+FN), TN/(TN+FP)
            delta = abs(TPR-TNR)
            if (delta < diff): 
                diff, opt_thld, opt_acc = delta, thld, acc

        opt_thlds.append(opt_thld)
        accs.append(opt_acc)

    print(f"...........................................val acc={np.mean(accs):.6f}")
    return np.mean(opt_thlds) 

def test(model, device, test_loader, thld=0.5):
    """test model"""
    model.eval()
    test_t0 = time()
    losses, accs = [], []
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader):
            data = data.to(device)
            output = model(data.x, data.edge_index, data.edge_attr)
            acc = binary_acc(y_pred = output, y_test = data.y, thld =  thld)
            loss = F.binary_cross_entropy(output.squeeze(1), data.y, 
                                          reduction='mean').item()
            accs.append(acc)
            losses.append(loss)
    
    #when batching works change acc for mean accs
    print(f"...testing time: {time()-test_t0}s")
    print(f'.............mean test loss={np.mean(losses):.6f}......test acc ={acc:.6f}\n')
    return np.mean(losses), np.mean(accs)

In [23]:
EPOCH = 200
model = InteractionNetwork(node_f_size = node_f_size, edge_attr_size = edge_attr_size,message_out = message_out, update_out= update_out,hidden_size = hidden_l_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.005)
scheduler = StepLR(optimizer, step_size=10,
                       gamma=0.9)       
from time import time
output = {'train_loss': [], 'test_loss': [], 'test_acc': []}
losses_l, accs_l,times_l = [],[],[]
for epoch in range(1, EPOCH + 1):
  print("---- Epoch {} ----".format(epoch))
  losses, accs,times = train( model, device, train_loader, optimizer, epoch)
  losses_l.append(losses)
  accs_l.append(accs)
  times_l.append(times)
  thld = validate(model, device, val_loader)
  print('...optimal threshold', thld)
  test_loss, test_acc = test(model, device, test_loader, thld=thld)
  scheduler.step()

---- Epoch 1 ----
...epoch time: 0.5256659984588623s
...epoch 1: mean train loss=0.727518......train acc=0.560408
...........................................val acc=0.718033
...optimal threshold 0.4917999999999999
...testing time: 0.5323059558868408s
.............mean test loss=0.535762......test acc =0.700559

---- Epoch 2 ----
...epoch time: 0.33099889755249023s
...epoch 2: mean train loss=0.355135......train acc=0.863308
...........................................val acc=0.900929
...optimal threshold 0.41600000000000004
...testing time: 0.35704898834228516s
.............mean test loss=0.191701......test acc =0.902386

---- Epoch 3 ----
...epoch time: 0.3202250003814697s
...epoch 3: mean train loss=0.218263......train acc=0.923618
...........................................val acc=0.874572
...optimal threshold 0.1506
...testing time: 0.35940098762512207s
.............mean test loss=0.191852......test acc =0.875887

---- Epoch 4 ----
...epoch time: 0.3310890197753906s
...epoch 4: mean

...testing time: 0.3625001907348633s
.............mean test loss=0.256581......test acc =0.926769

---- Epoch 28 ----
...epoch time: 0.33971405029296875s
...epoch 28: mean train loss=0.057184......train acc=0.978341
...........................................val acc=0.941635
...optimal threshold 0.0412
...testing time: 0.35855722427368164s
.............mean test loss=0.174693......test acc =0.942473

---- Epoch 29 ----
...epoch time: 0.34351277351379395s
...epoch 29: mean train loss=0.056925......train acc=0.978500
...........................................val acc=0.926838
...optimal threshold 0.06920000000000001
...testing time: 0.3609349727630615s
.............mean test loss=0.487075......test acc =0.927616

---- Epoch 30 ----
...epoch time: 0.3461019992828369s
...epoch 30: mean train loss=0.053374......train acc=0.980233
...........................................val acc=0.930478
...optimal threshold 0.08180000000000001
...testing time: 0.36406397819519043s
.............mean test l

...epoch time: 0.3388690948486328s
...epoch 54: mean train loss=0.035832......train acc=0.986618
...........................................val acc=0.958784
...optimal threshold 0.041999999999999996
...testing time: 0.3599050045013428s
.............mean test loss=0.108061......test acc =0.958125

---- Epoch 55 ----
...epoch time: 0.34082889556884766s
...epoch 55: mean train loss=0.035156......train acc=0.986656
...........................................val acc=0.946593
...optimal threshold 0.0066
...testing time: 0.36124205589294434s
.............mean test loss=0.115430......test acc =0.947806

---- Epoch 56 ----
...epoch time: 0.3406789302825928s
...epoch 56: mean train loss=0.034961......train acc=0.986774
...........................................val acc=0.951934
...optimal threshold 0.01
...testing time: 0.36231088638305664s
.............mean test loss=0.101251......test acc =0.954399

---- Epoch 57 ----
...epoch time: 0.34322524070739746s
...epoch 57: mean train loss=0.033928...

...testing time: 0.36766505241394043s
.............mean test loss=0.614284......test acc =0.960443

---- Epoch 81 ----
...epoch time: 0.34853100776672363s
...epoch 81: mean train loss=0.028941......train acc=0.988909
...........................................val acc=0.965722
...optimal threshold 0.0566
...testing time: 0.3664698600769043s
.............mean test loss=0.095700......test acc =0.970762

---- Epoch 82 ----
...epoch time: 0.34923219680786133s
...epoch 82: mean train loss=0.028869......train acc=0.989058
...........................................val acc=0.960547
...optimal threshold 0.023600000000000003
...testing time: 0.3685157299041748s
.............mean test loss=0.102489......test acc =0.964061

---- Epoch 83 ----
...epoch time: 0.3498058319091797s
...epoch 83: mean train loss=0.029121......train acc=0.988915
...........................................val acc=0.964228
...optimal threshold 0.028600000000000004
...testing time: 0.36580991744995117s
.............mean test

...........................................val acc=0.971179
...optimal threshold 0.0734
...testing time: 0.36925530433654785s
.............mean test loss=0.061199......test acc =0.970150

---- Epoch 108 ----
...epoch time: 0.35231685638427734s
...epoch 108: mean train loss=0.027035......train acc=0.989642
...........................................val acc=0.970440
...optimal threshold 0.058800000000000005
...testing time: 0.37026214599609375s
.............mean test loss=0.066067......test acc =0.972387

---- Epoch 109 ----
...epoch time: 0.35133910179138184s
...epoch 109: mean train loss=0.026833......train acc=0.989692
...........................................val acc=0.971387
...optimal threshold 0.0636
...testing time: 0.3687279224395752s
.............mean test loss=0.063349......test acc =0.970377

---- Epoch 110 ----
...epoch time: 0.3523252010345459s
...epoch 110: mean train loss=0.026578......train acc=0.989848
...........................................val acc=0.966978
...opti

...epoch time: 0.33963894844055176s
...epoch 134: mean train loss=0.024101......train acc=0.990819
...........................................val acc=0.963782
...optimal threshold 0.11099999999999999
...testing time: 0.42678403854370117s
.............mean test loss=0.150407......test acc =0.967226

---- Epoch 135 ----
...epoch time: 0.3427889347076416s
...epoch 135: mean train loss=0.023613......train acc=0.991006
...........................................val acc=0.965679
...optimal threshold 0.1136
...testing time: 0.3626289367675781s
.............mean test loss=0.146413......test acc =0.970067

---- Epoch 136 ----
...epoch time: 0.34616589546203613s
...epoch 136: mean train loss=0.023446......train acc=0.991135
...........................................val acc=0.973557
...optimal threshold 0.05920000000000001
...testing time: 0.35987091064453125s
.............mean test loss=0.063064......test acc =0.975239

---- Epoch 137 ----
...epoch time: 0.33896899223327637s
...epoch 137: mean 

...testing time: 0.3530576229095459s
.............mean test loss=0.074544......test acc =0.977026

---- Epoch 161 ----
...epoch time: 0.36878275871276855s
...epoch 161: mean train loss=0.022131......train acc=0.991683
...........................................val acc=0.964074
...optimal threshold 0.0922
...testing time: 0.3511359691619873s
.............mean test loss=0.168831......test acc =0.971521

---- Epoch 162 ----
...epoch time: 0.3420076370239258s
...epoch 162: mean train loss=0.022200......train acc=0.991571
...........................................val acc=0.974169
...optimal threshold 0.06420000000000001
...testing time: 0.352708101272583s
.............mean test loss=0.059230......test acc =0.974990

---- Epoch 163 ----
...epoch time: 0.34563493728637695s
...epoch 163: mean train loss=0.021984......train acc=0.991644
...........................................val acc=0.970663
...optimal threshold 0.049400000000000006
...testing time: 0.34903502464294434s
.............mean t

...epoch time: 0.3431248664855957s
...epoch 187: mean train loss=0.021117......train acc=0.992082
...........................................val acc=0.973192
...optimal threshold 0.0962
...testing time: 0.3648991584777832s
.............mean test loss=0.082101......test acc =0.975889

---- Epoch 188 ----
...epoch time: 0.3427000045776367s
...epoch 188: mean train loss=0.021228......train acc=0.991967
...........................................val acc=0.972247
...optimal threshold 0.049
...testing time: 0.3614499568939209s
.............mean test loss=0.080406......test acc =0.974855

---- Epoch 189 ----
...epoch time: 0.3405330181121826s
...epoch 189: mean train loss=0.020880......train acc=0.992047
...........................................val acc=0.969271
...optimal threshold 0.07200000000000001
...testing time: 0.36386919021606445s
.............mean test loss=0.104279......test acc =0.973139

---- Epoch 190 ----
...epoch time: 0.3518640995025635s
...epoch 190: mean train loss=0.02103

In [25]:
torch.save(model.state_dict(),  '../Models/model_master.pt')

In [27]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [28]:
get_n_params(model)

1624

In [29]:
list(model.parameters())

[Parameter containing:
 tensor([[-2.0241e-01,  7.0257e-01, -9.9243e-03,  4.3828e-02,  3.1206e-01,
           1.5523e-01, -1.1180e-01,  1.8410e+00,  1.3548e-02,  9.2349e-02,
          -6.8645e-01, -2.1767e-01, -1.1374e+01,  6.9185e-03,  2.4292e-01,
          -4.3965e-01,  6.2275e+00, -1.1280e+01,  1.5670e-03],
         [ 1.6756e-01,  7.0382e-02,  4.0425e-02, -2.1226e-01, -7.6354e-03,
          -1.2238e-01, -1.0758e-01,  3.3398e-01, -6.5571e-02,  3.3313e-01,
           1.3925e-02,  2.0542e-01, -4.4590e+00, -2.2441e-02,  1.4626e-01,
          -1.8587e+00, -8.2693e+00, -4.2226e+00, -7.9514e-01],
         [-4.0588e-02,  1.1289e-01, -8.3760e-02, -4.3455e-01, -7.1863e-02,
           3.3193e-04,  2.2656e-01,  1.8120e-01,  9.3369e-02,  5.5375e-01,
           1.5487e-01, -2.0748e-02,  1.5869e+00,  5.3894e-02, -6.7099e-02,
          -3.8548e+00, -9.0287e+00,  2.2133e+00,  7.8311e-01],
         [-1.4392e-01,  9.3208e-01, -1.1372e-02, -1.6603e-01,  4.4400e-02,
          -1.7829e-01,  3.5289e-01,  7

AttributeError: 'generator' object has no attribute 'shape'